In [ ]:
import numpy as np
from keras import models
from keras import layers
import pandas as pd
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error

# Set random seed
np.random.seed(0)
number_of_features = 7

In [ ]:
#china dataset that doesn't work
#df = pd.read_csv("preproc.csv") 
#train, test = train_test_split(df, test_size=0.2)
#train = train.drop(['Date','1d'], axis = 1)
#test = test.drop(['Date','1d'], axis = 1)

#train_y = train[train.columns[1]]
#train_x = train.drop(['PM2.5'], axis = 1)

#test_y = test[test.columns[1]]
#test_x = test.drop(['PM2.5'], axis = 1)




#example regression dataset that works
train_x, train_y = make_regression(n_samples = 10000,
                                       n_features = number_of_features,
                                       random_state = 0)

In [ ]:
def create_network(neurons = 1, lr = 0.0001):
    
    # Start neural network
    network = models.Sequential()

    # Add first hidden layer with a ReLU activation function
    network.add(layers.Dense(neurons, activation='relu', input_shape=(number_of_features,)))

    # Add second hidden layer with a ReLU activation function
    network.add(layers.Dense(neurons, activation='relu'))

    # Add final layer with a linear activation function
    network.add(layers.Dense(units=1), activation='linear')

    # Compile neural network
    network.compile(loss='mean_squared_error', optimizer='adam')
    
    # Return compiled network
    return network

In [ ]:
neural_network = KerasRegressor(build_fn=create_network, verbose=0)

In [ ]:
lr = [0.0001, 0.001, 0.01, 0.1, 0.3, 1, 0.2]
neurons = [1, 5, 10, 15, 20, 25, 30]

hyperparameters = dict(lr=lr, neurons = neurons)


In [ ]:
# Create grid search
grid = GridSearchCV(estimator=neural_network, cv=3, param_grid=hyperparameters)

# Fit grid search
grid_result = grid.fit(train_x, train_y)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))